# Pengembangan dan Pengoperasian Sistem Machine Learning - Muhammad Abiya Makruf

Melakukan import library yang dibutuhkan untuk keseluruhan proyek.

In [3]:
# Import library
import os
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

Melakukan set variabel seperti pipeline name, path untuk menyimpan output, path module, dan banyak lainnya.

In [4]:
# Nama pipeline
PIPELINE_NAME = "predicting-hiring-pipeline"

# Pipeline inputs 
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TRAINER_MODULE_FILE = "modules/trainer.py"
TUNER_MODULE_FILE = "modules/tuner.py"
COMPONENTS_MODULE_FILE = "modules/components.py"

# Pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

Pembuatan pipeline component module file menggunakan magic command. Pipeline terdiri dari:

    CsvExampleGen
    StatisticsGen
    SchemaGen
    ExampleValidator
    Transform
    Traine
    Tuner
    Evaluator
    Pusher

Komponen trainer sudah menggunakan komponen tuner. Pusher akan melakukan push model jika melebihi syarat dari BinaryAccuracy 0.8

In [14]:
%%writefile {COMPONENTS_MODULE_FILE}

"""
Author: abiyamf
Date: 1/8/2023
This is the components.py module.
Usage:
- ECreate a pipeline with TFX components.
"""

# Import library
import os
import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen, 
    StatisticsGen, 
    SchemaGen, 
    ExampleValidator, 
    Transform, 
    Trainer,
    Tuner,
    Evaluator,
    Pusher
)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2 
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy)

# Fungsi untuk melakukan inisialisasi components
def init_components(config):

    """Returns tfx components for the pipeline.
 
    Args:
        data_dir (str): Directory containing the dataset.
        transform_module (str): Path to the transform module.
        tuner_module (str): Path to the tuner module.
        training_module (str): Path to the training module.
        training_steps (int): Number of training steps.
        eval_steps (int): Number of evaluation steps.
        serving_model_dir (str): Directory to save the serving
 
    Returns:
        components: Tuple of TFX components.
    """ 
    
    # Membagi dataset dengan perbandingan 8:2
    output = example_gen_pb2.Output(
        split_config = example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
        ])
    )
 
    # Komponen example gen
    example_gen = CsvExampleGen(
        input_base=config["DATA_ROOT"], 
        output_config=output
    )
    
    # Komponen statistics gen
    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"]   
    )
    
    # Komponen schema gen
    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"]
    )
    
    # Komponen example validator
    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )
    
    # Komponen transform. Menggunakan module transform.py
    transform  = Transform(
        examples=example_gen.outputs['examples'],
        schema= schema_gen.outputs['schema'],
        module_file=os.path.abspath(config["transform_module"])
    )

    # Komponen tuner. Menggunakan module tuner.py
    tuner = Tuner(
        module_file=os.path.abspath(config["tuner_module"]),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(
            splits=['train'], 
            num_steps=config["training_steps"]),
        eval_args=trainer_pb2.EvalArgs(
            splits=['eval'], 
            num_steps=config["eval_steps"]),
    )
    
    # Komponen trainer. Menggunakan module trainer.py
    trainer  = Trainer(
        module_file=os.path.abspath(config["training_module"]),
        examples = transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        hyperparameters=tuner.outputs['best_hyperparameters'],
        train_args=trainer_pb2.TrainArgs(
            splits=['train'],
            num_steps=config["training_steps"]),
        eval_args=trainer_pb2.EvalArgs(
            splits=['eval'], 
            num_steps=config["eval_steps"])
    )
    
    # Komponen model resolver
    model_resolver = Resolver(
        strategy_class= LatestBlessedModelStrategy,
        model = Channel(type=Model),
        model_blessing = Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')
 
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value':0.8}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value':0.0001})
                        )
                )
            ])
    ]
 
    
    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='HiringDecision')],
        slicing_specs=[
            tfma.SlicingSpec(),
            ],
        metrics_specs=metrics_specs
    )
    
    # Komponen evaluator
    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config)
    
    # Komponen pusher
    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=config["serving_model_dir"]
            )
        ),
    )
    
    # Mengembalikan semua komponen
    components = (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        model_resolver,
        evaluator,
        pusher
    )
    
    # Mengembalikan komponen
    return components

Overwriting modules/components.py


Pembuatan module file transform menggunakan magic command. Module ini menambahkan "_xf" pada nama fitur dan melakukan normalisasi.

In [6]:
%%writefile {TRANSFORM_MODULE_FILE}

"""
Author: abiyamf
Date: 1/8/2023
This is the transfom.py module.
Usage:
- Preprocess input features into transformed features.
"""

# Import library
import tensorflow as tf
import tensorflow_transform as tft
 

# Daftar numerical fitur pada dataset
NUMERICAL_FEATURES = [
    "Age",
    "Gender",
    "EducationLevel",
    "ExperienceYears",
    "PreviousCompanies",
    "DistanceFromCompany",
    "InterviewScore",
    "SkillScore",
    "PersonalityScore",
    "RecruitmentStrategy",
]

# Label key
LABEL_KEY = "HiringDecision"
 
# Fungsi untuk mengubuah nama fitur yang sudah di transform
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 

# Fungsi untuk melakukan preprocessing
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    for feature in NUMERICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_0_1(inputs[feature])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting modules/transform.py


Pembuatan module file tuner menggunakan magic command. 

In [18]:
%%writefile {TUNER_MODULE_FILE}

"""
Author: abiyamf
Date: 1/8/2023
This is the trainer.py module.
Usage:
- Tuning the model.
"""

# Import library
import tensorflow as tf
import kerastuner as kt
import tensorflow_transform as tft
import keras_tuner as kt
from tfx.v1.components import TunerFnResult
from tfx.components.trainer.fn_args_utils import FnArgs
from trainer import NUMERICAL_FEATURES, transformed_name, input_fn


# Fungsi untuk membuat model
def model_builder(hyperparameters):
    """
    This function defines a Keras model and returns the model as a
    Keras object.
    """

    input_features = []

    for key in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(key))
        )

    concatenate = tf.keras.layers.concatenate(input_features)
    
    deep = tf.keras.layers.Dense(hyperparameters.Choice(
        'unit_1', [128, 256]), 
        activation="relu")(concatenate)
    deep = tf.keras.layers.Dropout(hyperparameters.Choice(
        'dropout_1', [0.2, 0.4]))(deep)

    deep = tf.keras.layers.Dense(hyperparameters.Choice(
        'unit_2', [64, 128]), 
        activation="relu")(deep)
    deep = tf.keras.layers.Dropout(hyperparameters.Choice(
        'dropout_2', [0.2, 0.4]))(deep)

    deep = tf.keras.layers.Dense(hyperparameters.Choice(
        'unit_3', [32, 64]), 
        activation="relu")(deep)
    deep = tf.keras.layers.Dropout(hyperparameters.Choice(
        'dropout_3', [0.2, 0.4]))(deep)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(deep)

    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hyperparameters.Choice(
                'learning_rate', [0.0001, 0.001])),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    return model

# Fungsi tuner
def tuner_fn(fn_args: FnArgs):
    """
    Tuning the model.
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, batch_size=10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, batch_size=10)
    
    tuner = kt.RandomSearch(
        model_builder,
        objective='val_accuracy',
        max_trials=15,
        directory=fn_args.working_dir,
        project_name='kt_hyperband'
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
            "epochs": 10
        }
    )

Overwriting modules/tuner.py


Pembuatan module file trainer menggunakan magic command.

In [8]:
%%writefile {TRAINER_MODULE_FILE}

# Import library

"""
Author: abiyamf
Date: 1/8/2023
This is the trainer.py module.
Usage:
- Define a Keras model.
"""

import os
import tensorflow as tf
from keras.utils.vis_utils import plot_model
import tensorflow_transform as tft 
 
from transform import (
    LABEL_KEY,
    NUMERICAL_FEATURES,
    transformed_name,
)
 
# Fungsi untuk membuat model
def get_model(show_summary=True):
    """
    This function defines a Keras model and returns the model as a
    Keras object.
    """
 
    # one-hot categorical features
    input_features = []
    
    for feature in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(feature))
        )
    
    concatenate = tf.keras.layers.concatenate(input_features)
    deep = tf.keras.layers.Dense(256, activation="relu")(concatenate)
    deep = tf.keras.layers.Dense(64, activation="relu")(deep)
    deep = tf.keras.layers.Dense(16, activation="relu")(deep)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(deep)
 
    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    if show_summary:
        model.summary()
 
    return model

# Fungsi untuk membaca data yang sudah di kompres
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
# Fungsi untuk mendapatkan fitur yang sudah di transform
def get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""
 
    model.tft_layer = tf_transform_output.transform_features_layer()
 
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec
        )
 
        transformed_features = model.tft_layer(parsed_features)
 
        outputs = model(transformed_features)
        return {"outputs": outputs}
 
    return serve_tf_examples_fn
 
# Fungsi untuk membuat dataset
def input_fn(file_pattern, tf_transform_output, batch_size=64):
    """Generates features and labels for tuning/training.
    Args:
        file_pattern: input tfrecord file pattern.
        tf_transform_output: A TFTransformOutput.
        batch_size: representing the number of consecutive elements of 
        returned dataset to combine in a single batch
    Returns:
        A dataset that contains (features, indices) tuple where features
        is a dictionary of Tensors, and indices is a single Tensor of
        label indices.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
 
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
    )
 
    return dataset
 
# Fungsi untuk menjalankan model
def run_fn(fn_args):
    """Train the model based on given args.
    Args:
    fn_args: Holds args used to train the model as name/value pairs.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
 
    train_dataset = input_fn(fn_args.train_files, tf_transform_output, 64)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, 64)
 
    model = get_model()
 
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )
 
    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback],
        epochs=10
    )
 
    signatures = {
        "serving_default": get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }
    model.save(
        fn_args.serving_model_dir, save_format="tf", signatures=signatures
    )
    
    plot_model(
        model, 
        to_file='images/model_plot.png', 
        show_shapes=True, 
        show_layer_names=True
    )

Overwriting modules/trainer.py


Melakukan inisialisasi local pipeline.

In [8]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available 
        # during execution time.
        "----direct_num_workers=0" 
    ]
    
    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

Menjalankan pipeline menggunakan Apache Beam.

In [9]:
from modules.components import init_components

logging.set_verbosity(logging.INFO)

config = {
    "DATA_ROOT": DATA_ROOT,
    "training_module": TRAINER_MODULE_FILE,
    "transform_module": TRANSFORM_MODULE_FILE,
    "tuner_module": TUNER_MODULE_FILE,
    "training_steps": 1000,
    "eval_steps": 250,
    "serving_model_dir": serving_model_dir,
}

components = init_components(config)

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: output\predicting-hiring-pipeline
c:\Users\ABIYA\anaconda3\envs\proyek-akhir-mlops\lib\site-packages\tfx\orchestration\pipeline.py:406: UserWarning: Node Trainer depends on the output of node Tuner, but Tuner is not included in the components of pipeline. Did you forget to add it?
  warnings.warn(
INFO:absl:Generating ephemeral wheel package for 'j:\\ABIYA\\TELKOM UNIVERSITY\\FOLDER MATA KULIAH\\SEMESTER 7\\Dicoding\\Dicoding-PengembanganDanPengoperasianSistemMachineLearning\\modules\\transform.py' (including modules: ['components', 'trainer', 'transform', 'tuner']).
INFO:absl:User module package has hash fingerprint version 037e05bae1ff8bbb36c8317a886635cca10dda1687efdf7d3b8a9ab30aa577d3.
INFO:absl:Executing: ['c:\\Users\\ABIYA\\anaconda3\\envs\\proyek-akhir-mlo

INFO:absl:Node CsvExampleGen depends on [].
INFO:absl:Node CsvExampleGen is scheduled.
INFO:absl:Node Latest_blessed_model_resolver depends on [].
INFO:absl:Node Latest_blessed_model_resolver is scheduled.
INFO:absl:Node StatisticsGen depends on ['Run[CsvExampleGen]'].
INFO:absl:Node StatisticsGen is scheduled.
INFO:absl:Node SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Node SchemaGen is scheduled.
INFO:absl:Node ExampleValidator depends on ['Run[SchemaGen]', 'Run[StatisticsGen]'].
INFO:absl:Node ExampleValidator is scheduled.
INFO:absl:Node Transform depends on ['Run[CsvExampleGen]', 'Run[SchemaGen]'].
INFO:absl:Node Transform is scheduled.
INFO:absl:Node Trainer depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Node Trainer is scheduled.
INFO:absl:Node Evaluator depends on ['Run[CsvExampleGen]', 'Run[Latest_blessed_model_resolver]', 'Run[Trainer]'].
INFO:absl:Node Evaluator is scheduled.
INFO:absl:Node Pusher depends on ['Run[Evaluator]', 'Run[Trainer]'].
INFO:absl

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.
INFO:absl:Feature Age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature DistanceFromCompany has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature EducationLevel has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ExperienceYears has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature HiringDecision has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature InterviewScore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature PersonalityScore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature PreviousCompanies has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature RecruitmentStrategy has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature SkillScore has a shape dim {
  size: 1
}
. Settin

INFO:tensorflow:Assets written to: output\predicting-hiring-pipeline\Transform\transform_graph\6\.temp_path\tftransform_tmp\0bdf64d370694839b547af1b77ec274a\assets


...

INFO:absl:Training complete. Model written to output\predicting-hiring-pipeline\Trainer\model\7\Format-Serving. ModelRun written to output\predicting-hiring-pipeline\Trainer\model_run\7
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 7 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "output\\predicting-hiring-pipeline\\Trainer\\model\\7"
, artifact_type: name: "Model"
base_type: MODEL
)], 'model_run': [Artifact(artifact: uri: "output\\predicting-hiring-pipeline\\Trainer\\model_run\\7"
, artifact_type: name: "ModelRun"
)]}) for execution 7
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Trainer is finished.
INFO:absl:node Evaluator is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.evaluator.component.Evaluator"
    base_type: EVALUATE
  }
  id: "Evaluator"
}
contexts {
  contexts {
    type {
    

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.8\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "HiringDecision"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}'} 'custom_extractors'
INFO:absl:Request was ma

INFO:absl:Evaluation complete. Results written to output\predicting-hiring-pipeline\Evaluator\evaluation\8.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to output\predicting-hiring-pipeline\Evaluator\blessing\8.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 8 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'evaluation': [Artifact(artifact: uri: "output\\predicting-hiring-pipeline\\Evaluator\\evaluation\\8"
, artifact_type: name: "ModelEvaluation"
)], 'blessing': [Artifact(artifact: uri: "output\\predicting-hiring-pipeline\\Evaluator\\blessing\\8"
, artifact_type: name: "ModelBlessing"
)]}) for execution 8
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Evaluator is finished.
INFO:absl:node Pusher is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.pusher.component.Pusher"
    base_type: DEPLOY
  }
  id: "Pusher"
}
contexts {
  contexts {
    type {
      nam